In [1]:
import pandas as pd
import numpy as np

In [2]:
# read the diabetic data to pandas dataframe
diabetes = pd.read_csv("diabetic_data.csv", low_memory =False)

## Data statistics

We'll first gather some statistical data based on our base csv file.
Data can be split into two categories:

* continuous variables (recorded data, based on tests)
* categorical variables (categories)

In [3]:
# Statistics for continuous variables
## 
print(f"Total observations is {len(diabetes)}. Total number of variables is {len(diabetes.columns)}")
## Time in hospital (days)
df_statistics = diabetes[["time_in_hospital", "num_lab_procedures", "num_procedures",
                    "num_medications","number_outpatient","number_inpatient",
                    "number_emergency", "number_diagnoses"]]
df_statistics.agg(['min','max','mean','median','std']).T

Total observations is 101766. Total number of variables is 50


,min,max,mean,median,std
time_in_hospital,1.0,14.0,4.395987,4.0,2.985108
num_lab_procedures,1.0,132.0,43.095641,44.0,19.674362
num_procedures,0.0,6.0,1.339730,1.0,1.705807
num_medications,1.0,81.0,16.021844,15.0,8.127566
number_outpatient,0.0,42.0,0.369357,0.0,1.267265
number_inpatient,0.0,21.0,0.635566,0.0,1.262863
number_emergency,0.0,76.0,0.197836,0.0,0.930472
number_diagnoses,1.0,16.0,7.422607,8.0,1.933600


In [4]:
# Statistics for categorical variables ("features")

## PS A1Cresult stands for HbA1c
## It's a test that tells you your average level of blood sugar over the past 2 to 3 months

col = [['race'], ['gender'], ['age'], ['readmitted'], ['A1Cresult'],['change']]
for i, col in enumerate(col):
    grouped = diabetes.groupby(col).agg({'race': {'Total': 'count', 'Percentage': lambda x: round(100*x.count()/len(diabetes),1)}})
    grouped.columns = grouped.columns.droplevel(0)
    
    print(grouped)
    print("="*50)

                 Total  Percentage
race                              
?                 2273         2.2
AfricanAmerican  19210        18.9
Asian              641         0.6
Caucasian        76099        74.8
Hispanic          2037         2.0
Other             1506         1.5
                 Total  Percentage
gender                            
Female           54708        53.8
Male             47055        46.2
Unknown/Invalid      3         0.0
          Total  Percentage
age                        
[0-10)      161         0.2
[10-20)     691         0.7
[20-30)    1657         1.6
[30-40)    3775         3.7
[40-50)    9685         9.5
[50-60)   17256        17.0
[60-70)   22483        22.1
[70-80)   26068        25.6
[80-90)   17197        16.9
[90-100)   2793         2.7
            Total  Percentage
readmitted                   
<30         11357        11.2
>30         35545        34.9
NO          54864        53.9
           Total  Percentage
A1Cresult                   
>

/Users/fathielshibani/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [5]:
# Densify age categories, new range is 20 years instead of 10
new_age = []

for row in diabetes["age"]:
    # Assign age groups [90-100) and [80-90) to new group [80-100)
    if row == "[90-100)":
        new_age.append("[80-100)")
    elif row == "[80-90)":
        new_age.append("[80-100)")
    # Assign age groups [70-80) and [60-70) to new group [60-80)
    elif row == "[70-80)":
        new_age.append("[60-80)")
    elif row == "[60-70)":
        new_age.append("[60-80)")
    # Assign age groups [50-60) and [40-50) to new group [40-60)
    elif row == "[50-60)":
        new_age.append("[40-60)")
    elif row == "[40-50)":
        new_age.append("[40-60)")
    # Assign age groups [30-40) and [20-30) to new group [20-40)
    elif row == "[30-40)":
        new_age.append("[20-40)")
    elif row == "[20-30)":
        new_age.append("[20-40)")
    # Assign age groups [10-20) and [0-10) to new group [0-20)
    elif row == "[10-20)":
        new_age.append("[0-20)")
    elif row == "[0-10)":
        new_age.append("[0-20)")
    # Assign missing values to "unknown"
    else:
        new_age.append("unknown")

# Add new age category column to dataframe
diabetes['new_age'] = new_age

In [6]:
# REMARK DUTCH
# Medications with one single result
## examide and citoglipton only have No as outcome. We can delete those columns
diabetes.drop(["examide", "citoglipton"], axis =1, inplace = True)

In [7]:
# REMARK DUTCH
# First, replace "?" values by NaN's
diabetes = diabetes.replace('?', np.NaN)

In [8]:
# REMARK DUTCH
# Number of patients with multiple inpatient visits
total = len(diabetes)
unique_patients = len(diabetes['patient_nbr'].unique())

print(f"Out of {total} rows, there are {unique_patients} unique patients")

Out of 101766 rows, there are 71518 unique patients


In [9]:
# REMARK DUTCH
# Only keep the patient's first visit
diabetes = diabetes.drop_duplicates(subset = 'patient_nbr', keep = 'first')
print(f"Number of first visits is {len(diabetes)}")

Number of first visits is 71518


In [10]:
# check for NULL values
diabetes.isnull().sum()

# check for duplicated rows
print(diabetes.shape)
print('There are ' + str(diabetes.duplicated().sum()) + ' duplicated rows')

(71518, 49)
There are 0 duplicated rows


In [11]:
# REMARK DUTCH, NO LONGER NEEDED AS WE REPLACE '?' WITH NaN
"""# check for missing values in non numeric columns
for col in diabetes.columns:
    if diabetes[col].dtype == object:
        count = 0
        count = [count + 1 for x in diabetes[col] if x == '?']
        print(col + ' ' + str(sum(count)))"""

"# check for missing values in non numeric columns\nfor col in diabetes.columns:\n    if diabetes[col].dtype == object:\n        count = 0\n        count = [count + 1 for x in diabetes[col] if x == '?']\n        print(col + ' ' + str(sum(count)))"

In [12]:
# REMARK DUTCH, WE MIGHT WANT TO KEEP MEDICAL_SPECIALTY
# drop columns with missing values (more than 40%)
diabetes = diabetes.drop(['weight', 'payer_code','medical_specialty'], axis=1)

In [13]:
# REMARK DUTCH, JUST LEAVE THIS, IT'S ARBITRARY
"""#fill the missing values (less than 3%) in Race column with most frequent value ('Caucasian' - 76%)
print('Race')
print(diabetes['race'].describe())
print(      )

diabetes.loc[:, 'race'].replace('?', 'Caucasian')"""

"#fill the missing values (less than 3%) in Race column with most frequent value ('Caucasian' - 76%)\nprint('Race')\nprint(diabetes['race'].describe())\nprint(      )\n\ndiabetes.loc[:, 'race'].replace('?', 'Caucasian')"

In [14]:
# REMARK DUTCH, DIAGNOSIS IS IMPORTANT DATA. LET'S JUST DELETE THE ROWS IF ALL DIAGNOSIS DATA IS MISSING
# Only drop diag_1, diag_2, diag_3 where all diagnosis data is missing
columns = ["diag_1", "diag_2", "diag_3"]
diabetes.dropna(subset = columns, how="all")

"""# delete rows from columns with missing values (less than 2%)
columns = ['diag_1','diag_2','diag_3']
for col in columns:
    diabetes = diabetes[diabetes[col] != '?']"""

"# delete rows from columns with missing values (less than 2%)\ncolumns = ['diag_1','diag_2','diag_3']\nfor col in columns:\n    diabetes = diabetes[diabetes[col] != '?']"

In [15]:
# prepare colums to convert into numeric columns
coulms = ['diag_1','diag_2','diag_3']
for col in columns:
    diabetes.loc[:, col].replace(to_replace=r'^V.$', value=0, regex=True)

In [16]:
# convert into numeric columns
coulms = ['diag_1','diag_2','diag_3']
for col in columns:
    diabetes[col] = pd.to_numeric(diabetes[col], errors='coerce')

In [17]:
## Use ICD9 codes to create primary diagnosis column from diag_1 column based on ICD9 codes categories

# create primary diagnosis column from diag_1 column
primary_diagnosis = []

for d in diabetes['diag_1']:
    if d >= 140 and d <= 239:
        primary_diagnosis.append('Neoplasms')    
    elif d >= 250.00 and d <= 250.99:
        primary_diagnosis.append('Diabetes')
    elif (d >= 390 and d <= 459) or d == 785:
        primary_diagnosis.append('Circulatory')
    elif (d >= 460 and d <= 519) or d == 786:
        primary_diagnosis.append('Respiratory')
    elif (d >= 520 and d <= 579) or d == 787:
        primary_diagnosis.append('Digestive')
    elif (d >= 580 and d <= 629) or d == 788:
        primary_diagnosis.append('Genitourinary')
    elif d >= 710 and d <= 739:
        primary_diagnosis.append('Musculoskeletal') 
    elif d >= 800 and d <= 999:
        primary_diagnosis.append('Injury')    
    else:
        primary_diagnosis.append('Other')
        
diabetes['primary diagnosis'] = primary_diagnosis    

            
# create primary diagnosis column from diag_2 column
second_diagnosis = []

for d in diabetes['diag_2']:
    if d >= 140 and d <= 239:
        second_diagnosis.append('Neoplasms')    
    elif d >= 250.00 and d <= 250.99:
        second_diagnosis.append('Diabetes')
    elif (d >= 390 and d <= 459) or d == 785:
        second_diagnosis.append('Circulatory')
    elif (d >= 460 and d <= 519) or d == 786:
        second_diagnosis.append('Respiratory')
    elif (d >= 520 and d <= 579) or d == 787:
        second_diagnosis.append('Digestive')
    elif (d >= 580 and d <= 629) or d == 788:
        second_diagnosis.append('Genitourinary')
    elif d >= 710 and d <= 739:
        second_diagnosis.append('Musculoskeletal') 
    elif d >= 800 and d <= 999:
        second_diagnosis.append('Injury')    
    else:
        second_diagnosis.append('Other')
        
diabetes['second diagnosis'] = second_diagnosis    


# create primary diagnosis column from diag_2 column
third_diagnosis = []

for d in diabetes['diag_3']:
    if d >= 140 and d <= 239:
        third_diagnosis.append('Neoplasms')    
    elif d >= 250.00 and d <= 250.99:
        third_diagnosis.append('Diabetes')
    elif (d >= 390 and d <= 459) or d == 785:
        third_diagnosis.append('Circulatory')
    elif (d >= 460 and d <= 519) or d == 786:
        third_diagnosis.append('Respiratory')
    elif (d >= 520 and d <= 579) or d == 787:
        third_diagnosis.append('Digestive')
    elif (d >= 580 and d <= 629) or d == 788:
        third_diagnosis.append('Genitourinary')
    elif d >= 710 and d <= 739:
        third_diagnosis.append('Musculoskeletal') 
    elif d >= 800 and d <= 999:
        third_diagnosis.append('Injury')    
    else:
        third_diagnosis.append('Other')
        
diabetes['third diagnosis'] = third_diagnosis                       

In [18]:
# create admission_type column from admission_type_id column ... for more details see IDs_mapping.csv file

admission_type = []

for a in diabetes['admission_type_id']:
    if a == 1:
        admission_type.append('Emergency')    
    elif a == 2:
        admission_type.append('Urgent')
    elif a == 3:
        admission_type.append('Elective')
    elif a == 4:
        admission_type.append('Newborn')
    elif a == 5:
        admission_type.append('Not Available')
    elif a == 6:
        admission_type.append('Unknown')
    elif a == 7:
        admission_type.append('Trauma Center')
    elif a == 8:
        admission_type.append('Not Mapped')    
            
diabetes['admission_type'] = admission_type 

In [19]:
diabetes['race'] = diabetes['race'].fillna('Caucasian')

In [20]:
# create label column from readmitted column (NO and >30 considered as 0, <30 as 1)
label = []

for l in diabetes['readmitted']:
    if l == 'NO'or l == '>30':
        label.append(0)    
    else:
        label.append(1)
           
diabetes['label'] = label

In [21]:
# REMARK DUTCH, YOU DON'T WANT TO OVERRIDE THE ORIGINAL FILE. SAVE RESULTS IN A NEW CSV
diabetes.to_csv("diabetes_cleaned.csv", index=False)